# Multi-Lifetime Metrics Evaluation

In [ ]:
# (c) 2019 The Johns Hopkins University Applied Physics Laboratory LLC (JHU/APL).
# All Rights Reserved. This material may be only be used, modified, or reproduced
# by or for the U.S. Government pursuant to the license rights granted under the
# clauses at DFARS 252.227-7013/7014 or FAR 52.227-14. For any other permission,
# please contact the Office of Technology Transfer at JHU/APL.

# NO WARRANTY, NO LIABILITY. THIS MATERIAL IS PROVIDED “AS IS.” JHU/APL MAKES NO
# REPRESENTATION OR WARRANTY WITH RESPECT TO THE PERFORMANCE OF THE MATERIALS,
# INCLUDING THEIR SAFETY, EFFECTIVENESS, OR COMMERCIAL VIABILITY, AND DISCLAIMS
# ALL WARRANTIES IN THE MATERIAL, WHETHER EXPRESS OR IMPLIED, INCLUDING (BUT NOT
# LIMITED TO) ANY AND ALL IMPLIED WARRANTIES OF PERFORMANCE, MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE, AND NON-INFRINGEMENT OF INTELLECTUAL PROPERTY
# OR OTHER THIRD PARTY RIGHTS. ANY USER OF THE MATERIAL ASSUMES THE ENTIRE RISK
# AND LIABILITY FOR USING THE MATERIAL. IN NO EVENT SHALL JHU/APL BE LIABLE TO ANY
# USER OF THE MATERIAL FOR ANY ACTUAL, INDIRECT, CONSEQUENTIAL, SPECIAL OR OTHER
# DAMAGES ARISING FROM THE USE OF, OR INABILITY TO USE, THE MATERIAL, INCLUDING,
# BUT NOT LIMITED TO, ANY DAMAGES FOR LOST PROFITS.

In [ ]:
# Import necessary modules
import json
from pathlib import Path

import matplotlib.pyplot as plt
import pandas as pd
import scipy
import seaborn as sns
from evaluation.evaluate import compute_metrics, load_computational_costs, save_ste_data

sns.set_style("dark")
sns.set_context("paper")

%matplotlib ipympl

In [ ]:
# Specify top-level directory for evaluation
# The path should be that of an agent configuration directory within an evaluation directory
# (e.g., m9_eval/agent_config-0/).
log_dir = Path("example_eval/m9_eval/agent_config-0")

In [ ]:
# Configure metrics report
perf_measure = 'performance'
transfer_method = 'both'
output = 'll_metrics.tsv'
do_smoothing = True
do_save = False

In [ ]:
# Load computational cost data
comp_cost_df = load_computational_costs(log_dir)
comp_cost_df

In [ ]:
# Save STE data in log directory
save_ste_data(log_dir)

In [ ]:
# Compute metrics for lifelong learning logs
ll_metrics_df = compute_metrics(log_dir, perf_measure, transfer_method, do_smoothing)

In [ ]:
# Show mean and standard deviation of data
ll_metrics_df.groupby(by=['complexity', 'difficulty']).agg(['mean', 'std'])

In [ ]:
# Show median and IQR of data
ll_metrics_df.groupby(by=['complexity', 'difficulty']).agg(['median', scipy.stats.iqr])

In [ ]:
# Plot aggregated data
fig = plt.figure(figsize=(12, 8))

for index, metric in enumerate(ll_metrics_df.drop(columns=['complexity', 'difficulty']).columns, start=1):
    # Create subplot for current metric
    ax = fig.add_subplot(3, 3, index)

    # Create grouped violin plot
    sns.violinplot(x='complexity', y=metric, hue='difficulty', data=ll_metrics_df, palette='muted')

    # Resize legend font
    plt.setp(ax.get_legend().get_title(), fontsize='8')
    plt.setp(ax.get_legend().get_texts(), fontsize='6')

fig.subplots_adjust(wspace=0.35, hspace=0.35)

In [ ]:
# Save the lifelong learning metrics DataFrame
if do_save:
    with open(output, 'w', newline='\n') as metrics_file:
        ll_metrics_df.to_csv(metrics_file, sep='\t')